<a href="https://colab.research.google.com/github/hadokan/python_basic2advance/blob/main/python_titanic_VS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sbn
# Download and read file from drive

from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd

Mounted at /content/gdrive


In [2]:



#Read csv file
data = pd.read_csv(r'/content/gdrive/My Drive/train.csv')



In [3]:
data.head(5)



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
ageX=data['Age']
embarkedX=data['Embarked']
ageX=ageX.fillna(data['Age'].mean())
embarkedX.fillna('S')
embarkedXD=pd.get_dummies(embarkedX)

In [5]:
data=data.drop(['Age','Cabin','Embarked'],axis=1)
data.isnull().sum()
data.shape

(891, 9)

In [6]:
train_hd=data.drop(['Survived','PassengerId','Ticket','Name'],axis=1)
train_hd.dtypes
train_hd.shape

(891, 5)

In [7]:
y=data['Survived']

In [8]:
pclass=pd.get_dummies(train_hd['Pclass'],)
sex=pd.get_dummies(train_hd['Sex'])
pclass = pclass.set_axis({'P1','P2','P3'}, axis=1)


In [9]:
train_hd_temp=train_hd.drop(['Pclass','Sex'],axis=1)

In [10]:
ageX.isnull().sum()

0

In [11]:
train_hd_new=pd.concat([train_hd_temp,pclass,sex,embarkedXD,ageX],axis=1)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train,X_test,y_train,y_test=train_test_split(train_hd_new,y,stratify=y,train_size=0.2,random_state=90)
#X_train,X_test,y_train,y_test=train_test_split(train_hd_new,y,train_size=0.25,random_state=90)

In [14]:
#lr=LogisticRegression(C=1,max_iter=10000)
#lr=LogisticRegression(solver='liblinear',C=1,max_iter=10000)
#from sklearn.svm import LinearSVC
#lr=LinearSVC()
#from sklearn.naive_bayes import GaussianNB
#lr=GaussianNB()

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)
scaler.fit(X_test)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)



from sklearn.tree import DecisionTreeClassifier
#-----------------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
#-----------------------------------------------------------------------
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier



gbrt=GradientBoostingClassifier(max_depth=1, learning_rate=0.01,random_state=0).fit(X_train,y_train)

dtc=DecisionTreeClassifier().fit(X_train,y_train)
log=LogisticRegression(max_iter=1000,solver='lbfgs').fit(X_train,y_train)
rnd=RandomForestClassifier(n_estimators=10).fit(X_train,y_train)
svm=SVC(gamma='auto').fit(X_train,y_train)

bag=BaggingClassifier(dtc,n_estimators=10,max_samples=0.8,n_jobs=-1,random_state=1).fit(X_train,y_train)

mlp=MLPClassifier(hidden_layer_sizes=[10],max_iter=1000,random_state=0).fit(X_train,y_train)

voting=VotingClassifier(estimators=[('lr',log),('rf',rnd),('svc',svm)],voting='hard').fit(X_train,y_train)
voting2=VotingClassifier(estimators=[('lr',log),('rf',rnd),('BG',bag)],voting='hard').fit(X_train,y_train)
voting3=VotingClassifier(estimators=[('GB',gbrt),('rf',rnd),('BG',bag)],voting='hard').fit(X_train,y_train)
voting4=VotingClassifier(estimators=[('lr',log),('GB',gbrt),('rf',rnd),('BG',bag)],voting='hard').fit(X_train,y_train)
voting5=VotingClassifier(estimators=[('MLP',mlp),('lr',log),('GB',gbrt),('rf',rnd),('BG',bag)],voting='hard').fit(X_train,y_train)

voting6=VotingClassifier(estimators=[('svc',svm),('lr',log),('GB',gbrt),('rf',rnd),('BG',bag)],voting='hard').fit(X_train,y_train)



In [15]:


dtc_train_perf=dtc.score(X_train,y_train).round(3)
dtc_test_perf=dtc.score(X_test,y_test).round(3)

bag_train_perf=bag.score(X_train,y_train).round(3)
bag_test_perf=bag.score(X_test,y_test).round(3)


log_train_perf=log.score(X_train,y_train).round(3)
log_test_perf=log.score(X_test,y_test).round(3)

rnd_train_perf=rnd.score(X_train,y_train).round(3)
rnd_test_perf=rnd.score(X_test,y_test).round(3)

svm_train_perf=svm.score(X_train,y_train).round(3)
svm_test_perf=svm.score(X_test,y_test).round(3)


gbrt_train_perf=gbrt.score(X_train,y_train).round(3)
gbrt_test_perf=gbrt.score(X_test,y_test).round(3)

mlp_train_perf=mlp.score(X_train,y_train).round(3)
mlp_test_perf=mlp.score(X_test,y_test).round(3)

voting_train_perf=voting.score(X_train,y_train).round(3)
voting_test_perf=voting.score(X_test,y_test).round(3)

voting2_train_perf=voting2.score(X_train,y_train).round(3)
voting2_test_perf=voting2.score(X_test,y_test).round(3)

voting3_train_perf=voting3.score(X_train,y_train).round(3)
voting3_test_perf=voting3.score(X_test,y_test).round(3)

voting4_train_perf=voting4.score(X_train,y_train).round(3)
voting4_test_perf=voting4.score(X_test,y_test).round(3)

voting5_train_perf=voting5.score(X_train,y_train).round(3)
voting5_test_perf=voting5.score(X_test,y_test).round(3)

voting6_train_perf=voting6.score(X_train,y_train).round(3)
voting6_test_perf=voting6.score(X_test,y_test).round(3)

print('DecisionTreeClassifier training performance is',dtc_train_perf)
print('DecisionTreeClassifier test performance is',dtc_test_perf)
print('------------------------------------------------------------')
print('BaggingClassifier training performance is',bag_train_perf)
print('BaggingClassifier test performance is',bag_test_perf)
print('------------------------------------------------------------')
print('LogisticRegression traing performance is',log_train_perf)
print('LogisticRegression test performance is',log_test_perf)
print('------------------------------------------------------------')
print('RandomForestClassifier traing performance is',rnd_train_perf)
print('RandomForestClassifier test performance is',rnd_test_perf)
print('------------------------------------------------------------')
print('SVC traing performance is',svm_train_perf)
print('SVC test performance is',svm_test_perf)
print('------------------------------------------------------------')
print('GradientBoostingClassifier traing performance is',gbrt_train_perf)
print('GradientBoostingClassifier test performance is',gbrt_test_perf)
print('------------------------------------------------------------')
print('MLPClassifier traing performance is',mlp_train_perf)
print('MLPClassifier test performance is',mlp_test_perf)


DecisionTreeClassifier training performance is 1.0
DecisionTreeClassifier test performance is 0.718
------------------------------------------------------------
BaggingClassifier training performance is 0.966
BaggingClassifier test performance is 0.75
------------------------------------------------------------
LogisticRegression traing performance is 0.854
LogisticRegression test performance is 0.776
------------------------------------------------------------
RandomForestClassifier traing performance is 1.0
RandomForestClassifier test performance is 0.763
------------------------------------------------------------
SVC traing performance is 0.854
SVC test performance is 0.769
------------------------------------------------------------
GradientBoostingClassifier traing performance is 0.837
GradientBoostingClassifier test performance is 0.774
------------------------------------------------------------
MLPClassifier traing performance is 0.893
MLPClassifier test performance is 0.777


In [16]:
print('------------------------------------------------------------')
print('Voting(LR-RND-SVM) traing performance is',voting_train_perf)
print('Voting(LR-RND-SVM) test performance is',voting_test_perf)
print('------------------------------------------------------------')
print('Voting(LR-RND-BG)  traing performance is',voting2_train_perf)
print('Voting(LR-RND-BG)  test performance is',voting2_test_perf)
print('------------------------------------------------------------')
print('Voting(BG-RND-GBRT)  traing performance is',voting3_train_perf)
print('Voting(BG-RND-GBRT)  test performance is',voting3_test_perf)
print('------------------------------------------------------------')
print('Voting(LR-BG-RND-GBRT)  traing performance is',voting4_train_perf)
print('Voting(LR-BG-RND-GBRT)  test performance is',voting4_test_perf)
print('------------------------------------------------------------')
print('Voting(MLP-LR-BG-RND-GBRT)  traing performance is',voting5_train_perf)
print('Voting(MLP-LR-BG-RND-GBRT)  test performance is',voting5_test_perf)
print('------------------------------------------------------------')
print('Voting(SVC-LR-BG-RND-GBRT)  traing performance is',voting6_train_perf)
print('Voting(SVC-LR-BG-RND-GBRT)  test performance is',voting6_test_perf)
print('------------------------------------------------------------')

------------------------------------------------------------
Voting(LR-RND-SVM) traing performance is 0.854
Voting(LR-RND-SVM) test performance is 0.769
------------------------------------------------------------
Voting(LR-RND-BG)  traing performance is 0.955
Voting(LR-RND-BG)  test performance is 0.787
------------------------------------------------------------
Voting(BG-RND-GBRT)  traing performance is 0.961
Voting(BG-RND-GBRT)  test performance is 0.771
------------------------------------------------------------
Voting(LR-BG-RND-GBRT)  traing performance is 0.899
Voting(LR-BG-RND-GBRT)  test performance is 0.781
------------------------------------------------------------
Voting(MLP-LR-BG-RND-GBRT)  traing performance is 0.899
Voting(MLP-LR-BG-RND-GBRT)  test performance is 0.79
------------------------------------------------------------
Voting(SVC-LR-BG-RND-GBRT)  traing performance is 0.86
Voting(SVC-LR-BG-RND-GBRT)  test performance is 0.773
----------------------------------